In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import re
sns.set(style="whitegrid")

df = pd.read_csv("../data/raw/surat_uncleaned.csv")

In [ ]:
def _norm(s: pd.Series) -> pd.Series:
    """Lowercase + strip + normalize whitespace. Keep NA."""
    return (s.astype("string")
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
              .str.lower())

def _canon_facing(s: pd.Series) -> pd.Series:
    t = _norm(s)
    t = t.str.replace("-", " ", regex=False)
    t = t.str.replace(r"\s+", " ", regex=True)  # IMPORTANT: normalize again

    conds = [
        t.str.contains(r"\bnorth east\b|\bne\b", na=False).to_numpy(dtype=bool),
        t.str.contains(r"\bnorth west\b|\bnw\b", na=False).to_numpy(dtype=bool),
        t.str.contains(r"\bsouth east\b|\bse\b", na=False).to_numpy(dtype=bool),
        t.str.contains(r"\bsouth west\b|\bsw\b", na=False).to_numpy(dtype=bool),
        t.str.contains(r"\bnorth\b", na=False).to_numpy(dtype=bool),
        t.str.contains(r"\bsouth\b", na=False).to_numpy(dtype=bool),
        t.str.contains(r"\beast\b",  na=False).to_numpy(dtype=bool),
        t.str.contains(r"\bwest\b",  na=False).to_numpy(dtype=bool),
    ]
    choices = ["NE", "NW", "SE", "SW", "N", "S", "E", "W"]

    out = np.select(conds, choices, default=None)
    return pd.Series(out, index=s.index, dtype="string")

def clean_categories_uncleaned_flats(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # ---------- Normalize raw categorical columns ----------
    area_raw = _norm(out.get("areaWithType", pd.Series([pd.NA]*len(out))))
    tx_raw   = _norm(out.get("transaction",  pd.Series([pd.NA]*len(out))))
    fur_raw  = _norm(out.get("furnishing",   pd.Series([pd.NA]*len(out))))
    fac_raw  = _norm(out.get("facing",       pd.Series([pd.NA]*len(out))))
    sta_raw  = _norm(out.get("status",       pd.Series([pd.NA]*len(out))))
    flo_raw  = _norm(out.get("floor",        pd.Series([pd.NA]*len(out))))

    # ---------- Vocabulary (allowed labels) ----------
    TX_OK  = {"new property", "resale"}
    FUR_OK = {"unfurnished", "semi-furnished", "furnished"}
    OWN_OK = {"freehold", "co-operative society", "cooperative society"}

    # ---------- (1) areaWithType -> area_type_clean ----------
    area_type_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    area_ok_mask = area_raw.isin({"super area", "carpet area", "built area", "plot area"})
    area_type_clean = area_type_clean.mask(area_ok_mask, area_raw)

    # ---------- Helper masks ----------
    tx_is_tx  = tx_raw.isin(TX_OK)
    tx_is_fur = tx_raw.isin(FUR_OK)
    tx_is_own = tx_raw.isin(OWN_OK)

    fur_is_fur = fur_raw.isin(FUR_OK)
    fur_is_tx  = fur_raw.isin(TX_OK)
    fur_is_own = fur_raw.isin(OWN_OK)

    sta_is_tx  = sta_raw.isin(TX_OK)
    sta_is_fur = sta_raw.isin(FUR_OK)
    sta_is_own = sta_raw.isin(OWN_OK)

    flo_is_tx  = flo_raw.isin(TX_OK)
    flo_is_fur = flo_raw.isin(FUR_OK)
    flo_is_own = flo_raw.isin(OWN_OK)

    # ---------- (2) transaction_clean (priority: transaction -> floor -> status -> furnishing) ----------
    transaction_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    transaction_clean = transaction_clean.mask(tx_is_tx, tx_raw)
    transaction_clean = transaction_clean.combine_first(flo_raw.where(flo_is_tx))
    transaction_clean = transaction_clean.combine_first(sta_raw.where(sta_is_tx))
    # salvage: if furnishing column accidentally contains TX
    transaction_clean = transaction_clean.combine_first(fur_raw.where(fur_is_tx))

    # ---------- (3) furnishing_clean (priority: furnishing -> transaction -> floor -> status) ----------
    furnishing_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    furnishing_clean = furnishing_clean.mask(fur_is_fur, fur_raw)
    furnishing_clean = furnishing_clean.combine_first(tx_raw.where(tx_is_fur))
    furnishing_clean = furnishing_clean.combine_first(flo_raw.where(flo_is_fur))   # IMPORTANT FIX
    furnishing_clean = furnishing_clean.combine_first(sta_raw.where(sta_is_fur))   # IMPORTANT FIX

    # if furnishing column accidentally contains TX/OWN, invalidate for furnishing_clean
    furnishing_clean = furnishing_clean.mask(fur_is_tx | fur_is_own, pd.NA)

    # ---------- (extra) ownership_clean ----------
    ownership_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    ownership_clean = ownership_clean.combine_first(sta_raw.where(sta_is_own))
    ownership_clean = ownership_clean.combine_first(flo_raw.where(flo_is_own))
    ownership_clean = ownership_clean.combine_first(tx_raw.where(tx_is_own))
    ownership_clean = ownership_clean.combine_first(fur_raw.where(fur_is_own))

    # ---------- (4) facing_clean + view_clean ----------
    view_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    is_view = fac_raw.str.contains(r"(?:main road|garden|park|pool)", na=False)
    view_clean = view_clean.mask(is_view, fac_raw)

    is_numeric_only = fac_raw.str.fullmatch(r"\d+", na=False)

    facing_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    facing_clean = facing_clean.combine_first(_canon_facing(fac_raw))
    facing_clean = facing_clean.mask(is_view | is_numeric_only, pd.NA)

    # fallback from other contaminated cols (only fill when missing)
    facing_clean = facing_clean.combine_first(_canon_facing(fur_raw))
    facing_clean = facing_clean.combine_first(_canon_facing(tx_raw))
    facing_clean = facing_clean.combine_first(_canon_facing(sta_raw))
    facing_clean = facing_clean.combine_first(_canon_facing(flo_raw))

    # ---------- (5) status -> ready_to_move + possession ----------
    is_ready_to_move = sta_raw.str.contains(r"ready to move", na=False)

    poss = sta_raw.str.extract(
        r"(?:poss\.?\s*by|possession\s*by)\s*([a-z]{3,9})\s*'(\d{2})",
        expand=True
    )
    poss_month = poss[0].astype("string")
    poss_year2 = poss[1].astype("string")

    # IMPORTANT FIX: avoid producing 20 when year is missing
    year_str = np.where(poss_year2.notna(), "20" + poss_year2, np.nan)
    possession_year = pd.to_numeric(year_str, errors="coerce")

    # Cross-fill TX/OWN if status contains them (already covered above but harmless)
    transaction_clean = transaction_clean.combine_first(sta_raw.where(sta_is_tx))
    ownership_clean   = ownership_clean.combine_first(sta_raw.where(sta_is_own))

    # ---------- (6) floor -> floor_num, total_floors (+ cross-fill) ----------
    floor_num = pd.Series(np.nan, index=out.index, dtype="float")
    total_floors = pd.Series(np.nan, index=out.index, dtype="float")

    g = flo_raw.str.extract(r"^ground\s*out\s*of\s*(\d+)$", expand=True)
    g_total = pd.to_numeric(g[0], errors="coerce")

    m = flo_raw.str.extract(r"(\d+)\s*out\s*of\s*(\d+)", expand=True)
    m_floor = pd.to_numeric(m[0], errors="coerce")
    m_total = pd.to_numeric(m[1], errors="coerce")

    floor_num = floor_num.combine_first(m_floor)
    total_floors = total_floors.combine_first(m_total)

    floor_num = floor_num.mask(g_total.notna(), 0.0)
    total_floors = total_floors.combine_first(g_total)

    # if floor column is actually TX/OWN/FUR → salvage + nullify floor parse
    transaction_clean = transaction_clean.combine_first(flo_raw.where(flo_is_tx))
    ownership_clean   = ownership_clean.combine_first(flo_raw.where(flo_is_own))
    furnishing_clean  = furnishing_clean.combine_first(flo_raw.where(flo_is_fur))  # IMPORTANT FIX

    bad_floor = flo_is_tx | flo_is_own | flo_is_fur
    floor_num = floor_num.mask(bad_floor, np.nan)          # IMPORTANT FIX
    total_floors = total_floors.mask(bad_floor, np.nan)    # IMPORTANT FIX

    floor_ratio = floor_num / total_floors

    # ---------- Optional: parking extraction ----------
    parking_clean = pd.Series(pd.NA, index=out.index, dtype="string")
    is_parking = fur_raw.str.contains(r"\b(?:covered|open)\b", na=False) & fur_raw.str.contains(r"\d", na=False)
    parking_clean = parking_clean.mask(is_parking, fur_raw)

    # ---------- Attach clean columns ----------
    out["area_type_clean"] = area_type_clean
    out["transaction_clean"] = transaction_clean
    out["furnishing_clean"] = furnishing_clean
    out["ownership_clean"] = ownership_clean
    out["facing_clean"] = facing_clean
    out["view_clean"] = view_clean

    out["is_ready_to_move"] = is_ready_to_move
    out["possession_month"] = poss_month
    out["possession_year"]  = possession_year

    out["floor_num"] = floor_num
    out["total_floors"] = total_floors
    out["floor_ratio"] = floor_ratio

    out["parking_clean"] = parking_clean

    return out


In [13]:
df_cat = clean_categories_uncleaned_flats(df)
df_cat[[
    "areaWithType","area_type_clean",
    "transaction","transaction_clean",
    "furnishing","furnishing_clean",
    "facing","facing_clean","view_clean",
    "status","is_ready_to_move","possession_month","possession_year",
    "floor","floor_num","total_floors","floor_ratio",
    "ownership_clean","parking_clean"
]].head(10)

C:\Users\MY PC\AppData\Local\Temp\ipykernel_9240\3844610978.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_view = fac_raw.str.contains(r"(main road|garden|park|pool)", na=False)
C:\Users\MY PC\AppData\Local\Temp\ipykernel_9240\3844610978.py:137: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  floor_num = floor_num.combine_first(m_floor)
C:\Users\MY PC\AppData\Local\Temp\ipykernel_9240\3844610978.py:138: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  total_floor

,areaWithType,area_type_clean,transaction,transaction_clean,furnishing,furnishing_clean,facing,facing_clean,view_clean,status,is_ready_to_move,possession_month,possession_year,floor,floor_num,total_floors,floor_ratio,ownership_clean,parking_clean
0,Carpet Area,carpet area,New Property,new property,Unfurnished,unfurnished,West,W,<NA>,Poss. by Oct '24,False,oct,2024.0,5 out of 10,5,10,0.5,<NA>,<NA>
1,Super Area,super area,New Property,new property,Unfurnished,unfurnished,South -West,SW,<NA>,Poss. by Jan '26,False,jan,2026.0,6 out of 14,6,14,0.428571,<NA>,<NA>
2,Super Area,super area,Resale,resale,Semi-Furnished,semi-furnished,East,E,<NA>,Ready to Move,True,<NA>,NaN,5 out of 13,5,13,0.384615,<NA>,<NA>
3,Carpet Area,carpet area,New Property,new property,Unfurnished,unfurnished,East,E,<NA>,Ready to Move,True,<NA>,NaN,6 out of 14,6,14,0.428571,<NA>,<NA>
4,Super Area,super area,Orchid Fantasia,new property,2,unfurnished,2,<NA>,<NA>,New Property,False,<NA>,NaN,Unfurnished,<NA>,<NA>,<NA>,<NA>,<NA>
5,Super Area,super area,Unfurnished,new property,Anand Aspire,unfurnished,2,<NA>,<NA>,Poss. by Dec '25,False,dec,2025.0,New Property,<NA>,<NA>,<NA>,<NA>,<NA>
6,Carpet Area,carpet area,Unfurnished,new property,East,unfurnished,Main Road,E,main road,Poss. by Jun '24,False,jun,2024.0,New Property,<NA>,<NA>,<NA>,<NA>,<NA>
7,Carpet Area,carpet area,New Property,new property,1,<NA>,NaN,<NA>,<NA>,Poss. by Sep '25,False,sep,2025.0,7 out of 10,7,10,0.7,<NA>,<NA>
8,Super Area,super area,Orchid Gardenia,new property,2,unfurnished,2,<NA>,<NA>,New Property,False,<NA>,NaN,Unfurnished,<NA>,<NA>,<NA>,<NA>,<NA>
9,Carpet Area,carpet area,New Property,new property,Unfurnished,unfurnished,East,E,<NA>,Poss. by Dec '25,False,dec,2025.0,3 out of 14,3,14,0.214286,<NA>,<NA>


In [14]:
def quick_check_clean(df_cat):
    cols = ["area_type_clean","transaction_clean","furnishing_clean","facing_clean","view_clean","ownership_clean"]
    for c in cols:
        print(f"\n== {c} ==")
        print(df_cat[c].value_counts(dropna=False).head(15))

    print("\nMissing % (clean cols):")
    display((df_cat[cols].isna().mean()*100).round(2).sort_values(ascending=False))

quick_check_clean(df_cat)


== area_type_clean ==
area_type_clean
super area     2599
carpet area    1671
plot area       214
built area       35
<NA>              6
Name: count, dtype: Int64

== transaction_clean ==
transaction_clean
resale          2723
new property    1775
<NA>              27
Name: count, dtype: Int64

== furnishing_clean ==
furnishing_clean
unfurnished       2736
<NA>               683
semi-furnished     580
furnished          526
Name: count, dtype: Int64

== facing_clean ==
facing_clean
<NA>    1835
E       1735
SW       362
NE       248
W        109
N        108
SE        58
S         41
NW        29
Name: count, dtype: Int64

== view_clean ==
view_clean
<NA>                                         4113
main road                                     184
garden/park, main road                        124
garden/park                                    68
pool, garden/park, main road                   12
garden/park, pool, main road                   11
garden/park, pool                      

ownership_clean      96.62
view_clean           90.90
facing_clean         40.55
furnishing_clean     15.09
transaction_clean     0.60
area_type_clean       0.13
dtype: float64